In [2]:
import pandas as pd
import numpy as np


df = pd.read_csv('spotify_tracks.csv')


df.head()

,track_id,track_name,artist_name,year,popularity,artwork_url,album_name,acousticness,danceability,duration_ms,...,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,track_url,language
0,2r0ROhr7pRN4MXDMT1fEmd,"Leo Das Entry (From ""Leo"")",Anirudh Ravichander,2024,59,https://i.scdn.co/image/ab67616d0000b273ce9c65...,"Leo Das Entry (From ""Leo"")",0.0241,0.753,97297.0,...,8.0,0.1000,-5.994,0.0,0.1030,110.997,4.0,0.459,https://open.spotify.com/track/2r0ROhr7pRN4MXD...,Tamil
1,4I38e6Dg52a2o2a8i5Q5PW,AAO KILLELLE,"Anirudh Ravichander, Pravin Mani, Vaishali Sri...",2024,47,https://i.scdn.co/image/ab67616d0000b273be1b03...,AAO KILLELLE,0.0851,0.780,207369.0,...,10.0,0.0951,-5.674,0.0,0.0952,164.995,3.0,0.821,https://open.spotify.com/track/4I38e6Dg52a2o2a...,Tamil
2,59NoiRhnom3lTeRFaBzOev,Mayakiriye Sirikiriye - Orchestral EDM,"Anirudh Ravichander, Anivee, Alvin Bruno",2024,35,https://i.scdn.co/image/ab67616d0000b27334a1dd...,Mayakiriye Sirikiriye (Orchestral EDM),0.0311,0.457,82551.0,...,2.0,0.0831,-8.937,0.0,0.1530,169.996,4.0,0.598,https://open.spotify.com/track/59NoiRhnom3lTeR...,Tamil
3,5uUqRQd385pvLxC8JX3tXn,Scene Ah Scene Ah - Experimental EDM Mix,"Anirudh Ravichander, Bharath Sankar, Kabilan, ...",2024,24,https://i.scdn.co/image/ab67616d0000b27332e623...,Scene Ah Scene Ah (Experimental EDM Mix),0.2270,0.718,115831.0,...,7.0,0.1240,-11.104,1.0,0.4450,169.996,4.0,0.362,https://open.spotify.com/track/5uUqRQd385pvLxC...,Tamil
4,1KaBRg2xgNeCljmyxBH1mo,Gundellonaa X I Am A Disco Dancer - Mashup,"Anirudh Ravichander, Benny Dayal, Leon James, ...",2024,22,https://i.scdn.co/image/ab67616d0000b2735a59b6...,Gundellonaa X I Am a Disco Dancer (Mashup),0.0153,0.689,129621.0,...,7.0,0.3450,-9.637,1.0,0.1580,128.961,4.0,0.593,https://open.spotify.com/track/1KaBRg2xgNeCljm...,Tamil


In [6]:
import base64
import requests

CLIENT_ID = 'ad38583c4e9d439089af13c69d69a461'
CLIENT_SECRET = 'e1877d5002e44bd582a71b3d48f7e64c'  # 从开发者面板获取

# 准备获取token的请求
auth_url = 'https://accounts.spotify.com/api/token'
auth_header = base64.b64encode(f"{CLIENT_ID}:{CLIENT_SECRET}".encode()).decode()
headers = {
    'Authorization': f'Basic {auth_header}'
}
data = {
    'grant_type': 'client_credentials'
}

# 发送请求获取token
try:
    auth_response = requests.post(auth_url, headers=headers, data=data)
    auth_response.raise_for_status()  # 检查请求是否成功
    auth_data = auth_response.json()
    access_token = auth_data['access_token']
    print("Successfully obtained access token")
except Exception as e:
    print(f"Error obtaining access token: {str(e)}")
    raise

# 现在可以使用这个token来调用Spotify API
HEADERS = {
    "Authorization": f"Bearer {access_token}"
}

print(access_token)

Successfully obtained access token
BQD-F75ZF1qSXHL8qW98yxQU2txQheGSQKh9yo13mqziDtS6nIM8oPAkLNP0o6DK6rkfnHEnywMLZNxjLwcaEYy-z_OnCo7_ZfGAdUGhhkQ-nckwwp4


In [7]:
!pip install requests
!pip install tqdm
import requests
import time
from tqdm import tqdm  # 用于显示进度条

# Spotify API 配置
SPOTIFY_TOKEN = access_token
HEADERS = {
    "Authorization": f"Bearer {SPOTIFY_TOKEN}"
}

# 读取CSV文件
df = pd.read_csv('spotify_tracks.csv')

# 创建新的release_date列
df['release_date'] = None

# 处理每个track
for index, row in tqdm(df.iterrows(), total=len(df)):
    try:
        # 获取track信息
        track_response = requests.get(
            f"https://api.spotify.com/v1/tracks/{row['track_id']}", 
            headers=HEADERS
        )
        track_data = track_response.json()
        
        # 获取album_id
        album_id = track_data['album']['id']
        
        # 获取album信息
        album_response = requests.get(
            f"https://api.spotify.com/v1/albums/{album_id}", 
            headers=HEADERS
        )
        album_data = album_response.json()
        
        # 保存release_date
        df.at[index, 'release_date'] = album_data['release_date']
        
        # 避免触发API速率限制
        time.sleep(0.5)  # 每次请求后暂停0.5秒
        
    except Exception as e:
        print(f"Error processing track {row['track_id']}: {str(e)}")
        continue

# 保存更新后的CSV文件
df.to_csv('spotify_tracks_with_release_date.csv', index=False)

  4%|▍         | 2503/62317 [32:15<7:27:36,  2.23it/s] 

Error processing track 2HPVvTJ5BZGLOXTyrPH3wy: Expecting value: line 1 column 1 (char 0)
Error processing track 415We0rxvC1bJ9yzXyfBa9: Expecting value: line 1 column 1 (char 0)
Error processing track 6mCBXs5VNOS3OuiKrtyWxT: Expecting value: line 1 column 1 (char 0)


  4%|▍         | 2507/62317 [32:16<3:42:57,  4.47it/s]

Error processing track 3pwNBn7nc0cuPyGWCRL617: Expecting value: line 1 column 1 (char 0)
Error processing track 2kAZP4MIGCitpBHn358pgU: Expecting value: line 1 column 1 (char 0)
Error processing track 7rto8ANj8esy1YSfDekGSL: Expecting value: line 1 column 1 (char 0)


  4%|▍         | 2509/62317 [32:16<2:55:29,  5.68it/s]

...

 32%|███▏      | 19771/62317 [57:20<1:00:38, 11.69it/s]

Error processing track 3HbFdIHfJTU68z1WFwQL5h: Expecting value: line 1 column 1 (char 0)
Error processing track 5RaxRE1uVL5iWJyph5kHAM: Expecting value: line 1 column 1 (char 0)
Error processing track 0798zdrJufYfWMXYMJus5W: Expecting value: line 1 column 1 (char 0)


 32%|███▏      | 19775/62317 [57:20<1:02:00, 11.43it/s]

Error processing track 6nTraG97H0XX81xee4uvz3: Expecting value: line 1 column 1 (char 0)
Error processing track 37acTHSE33fh03njpyhq59: Expecting value: line 1 column 1 (char 0)
Error processing track 10itT98dhdiKQBt11j1ZWB: Expecting value: line 1 column 1 (char 0)
Error processing track 4HMeewaVu7PughGs52fDDe: Expecting value: line 1 column 1 (char 0)


 32%|███▏      | 19775/62317 [1:32:14<1:02:00, 11.43it/s]